In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import os

# Define dataset paths
train_dir = "D:/Major Project/normalized/train"
valid_dir = "D:/Major Project/normalized/valid"

# Define image size and batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 16  # Reducing batch size allows more frequent weight updates

# Apply data augmentation to the training set to enhance model generalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Rescale the validation set without augmentation
valid_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories and apply preprocessing
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Define the CNN model with additional feature extraction and regularization techniques
model = Sequential([
    # First convolutional block
    Conv2D(32, (3,3), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Second convolutional block
    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Third convolutional block
    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Additional convolutional layer for better feature extraction
    Conv2D(256, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    # Flatten layer converts feature maps into a one-dimensional vector
    Flatten(),
    
    # Fully connected dense layer with reduced number of neurons to prevent overfitting
    Dense(128, activation='relu', kernel_regularizer=l2(0.001)),

    # Dropout regularization to improve generalization
    Dropout(0.6),  

    # Output layer with softmax activation for multi-class classification
    Dense(4, activation='softmax')
])

# Compile the model with an adaptive optimizer and loss function for multi-class classification
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Apply early stopping to prevent overfitting and retain the best model
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Verify image batches before training
batch_x, batch_y = next(train_generator)
print(f"Batch Shape (Images): {batch_x.shape}")
print(f"Batch Shape (Labels): {batch_y.shape}")

# Train the model with the optimized parameters
EPOCHS = 25
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=EPOCHS,
    callbacks=[early_stop]  
)

# Save the trained model
model.save("models/custom_cnn_teeth_health_v3.h5")

# Print the model architecture
model.summary()


Found 16000 images belonging to 4 classes.
Found 4121 images belonging to 4 classes.


C:\Users\saite\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Batch Shape (Images): (16, 224, 224, 3)
Batch Shape (Labels): (16, 4)
Epoch 1/25


C:\Users\saite\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 542s 540ms/step - accuracy: 0.2912 - loss: 1.9544 - val_accuracy: 0.4045 - val_loss: 1.7645
Epoch 2/25
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 539s 538ms/step - accuracy: 0.3483 - loss: 1.5689 - val_accuracy: 0.3868 - val_loss: 1.7884
Epoch 3/25
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 541s 541ms/step - accuracy: 0.3621 - loss: 1.5363 - val_accuracy: 0.4545 - val_loss: 1.5437
Epoch 4/25
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 541s 540ms/step - accuracy: 0.3618 - loss: 1.5073 - val_accuracy: 0.2536 - val_loss: 2.0755
Epoch 5/25
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 541s 540ms/step - accuracy: 0.3669 - loss: 1.4783 - val_accuracy: 0.4215 - val_loss: 1.5179
Epoch 6/25
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 544s 543ms/step - accuracy: 0.3865 - loss: 1.4396 - val_accuracy: 0.4266 - val_loss: 1.6962
Epoch 7/25
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 542s 542ms/step - accuracy: 0.3913 - loss: 1.4157 - val_accuracy: 0.4511 - val_loss: 1.3951
Epoch 8/25
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 542s 541ms/step - accuracy: 0.3

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 222, 222, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 109, 109, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 52, 52, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 24, 24, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 24, 24, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 12, 12, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 36864)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     4,718,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,326,798 (58.47 MB)

 Trainable params: 5,108,612 (19.49 MB)

 Non-trainable params: 960 (3.75 KB)

 Optimizer params: 10,217,226 (38.98 MB)